In [11]:
import sys

sys.path.append("../../src")

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.linalg import eigh
import sto_ng
import sto_ng_overlap_function
import h5py
import gaussian_overlap_function


importlib.reload(sto_ng)
importlib.reload(sto_ng_overlap_function)
importlib.reload(gaussian_overlap_function)
from sto_ng import *
from sto_ng_overlap_function import *
from gaussian_overlap_function import *

In [13]:
with h5py.File("../make_base/sto_ng.h5", "r") as f:
    alphas = f["1s/sto-3g/alpha"][:]
    cs = f["1s/sto-3g/c"][:]
alphas = np.array(alphas)
cs = np.array(cs)
cs_new = cs * (2 * alphas / np.pi) ** (3 / 4)

In [14]:
def get_smat(stong, zetas, Rs):
    s_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            s_mat[i, j] = stong.norm(zetas[i], zetas[j], Rs[i], Rs[j])
    return s_mat

In [15]:
def get_kinetic_mat(stong, zetas, Rs):
    kinetic_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            kinetic_mat[i, j] = stong.kinetic(zetas[i], zetas[j], Rs[i], Rs[j])
    return kinetic_mat

In [16]:
def get_potential_mat_0(stong, zetas, Rs):
    potential_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            potential_mat[i, j] = 2 * stong.coulomb_ion(
                zetas[i], zetas[j], Rs[i], Rs[j], Rs[0]
            )
    return potential_mat


def get_potential_mat_1(stong, zetas, Rs):
    potential_mat = np.zeros((2, 2))
    for i in range(2):
        for j in range(2):
            potential_mat[i, j] = stong.coulomb_ion(
                zetas[i], zetas[j], Rs[i], Rs[j], Rs[1]
            )
    return potential_mat


def get_potential_mat(stong, zetas, Rs):
    return get_potential_mat_0(stong, zetas, Rs) + get_potential_mat_1(stong, zetas, Rs)

In [17]:
def get_ee_coulomb_mat(stong, zetas, Rs):
    ee_coulomb_mat = np.zeros((2, 2, 2, 2))
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    ee_coulomb_mat[i, j, k, l] = stong.coulomb_ee(
                        zetas[i],
                        zetas[j],
                        zetas[k],
                        zetas[l],
                        Rs[i],
                        Rs[j],
                        Rs[k],
                        Rs[l],
                    )
    return ee_coulomb_mat

In [18]:
def get_h_mat(stong, zetas, Rs):
    kinetic_mat = get_kinetic_mat(stong, zetas, Rs)
    potential_mat = get_potential_mat(stong, zetas, Rs)
    return kinetic_mat + potential_mat


def get_2e_mat(stong, zetas, Rs, c_sto3g):
    ee_coulomb_mat = get_ee_coulomb_mat(stong, zetas, Rs)
    ee_coulomb_mat_1 = np.tensordot(c_sto3g, ee_coulomb_mat, axes=(0, 1))
    ee_coulomb_mat_13 = np.tensordot(c_sto3g, ee_coulomb_mat_1, axes=(0, 2))
    return ee_coulomb_mat_13


# def get_fock_mat(stong, zetas, Rs, c_sto3g):
#    kinetic_mat = get_kinetic_mat(cs, alphabetas, Rs)
#    potential_mat = get_potential_mat(cs, alphabetas, Rs)
#    ee_coulomb_mat = c_chi @ get_ee_coulomb_mat(cs, alphabetas, Rs) @ c_chi
#    fock_mat = kinetic_mat + potential_mat + ee_coulomb_mat
#    return fock_mat

In [19]:
stong = StoNG(cs_new, alphas)
zetaHe = 2.0925
zetaH = 1.24
Rs = [np.array([0, 0, 0]), np.array([0, 0, 1.4632])]
zetas = [zetaHe, zetaH]
c_sto3g = np.array([1, 0.1])
s_mat = get_smat(stong, zetas, Rs)
h_mat = get_h_mat(stong, zetas, Rs)
J_mat = get_2e_mat(stong, zetas, Rs, c_sto3g)
fock_mat = h_mat + J_mat
for i in range(100):
    s_mat = get_smat(stong, zetas, Rs)
    h_mat = get_h_mat(stong, zetas, Rs)
    J_mat = get_2e_mat(stong, zetas, Rs, c_sto3g)
    fock_mat = h_mat + J_mat
    val, vec = eigh(fock_mat, s_mat)
    c_sto3g = 0.5 * c_sto3g + 0.5 * vec[:, 0]
    print(val[0] * 2 - vec[:, 0] @ J_mat @ vec[:, 0])

-5.187488887326078
-5.228161049706241
-4.8274296808431485
-4.532937165717043
-4.375122482275911
-4.29884787920455
-4.262404819429457
-4.244750009789701
-4.236086115244381
-4.23179969281356
-4.229669091019651
-4.228607290727727
-4.2280773660333235
-4.22781267650171
-4.227680408295445
-4.227614295742629
-4.227581245548979
-4.227564722173309
-4.227556460973567
-4.227552330512385
-4.227550265321273
-4.227549232736971
-4.227548716448034
-4.227548458304485
-4.227548329232972
-4.22754826469729
-4.227548232429473
-4.227548216295568
-4.22754820822862
-4.227548204195144
-4.227548202178408
-4.2275482011700385
-4.227548200665854
-4.227548200413763
-4.227548200287717
-4.227548200224694
-4.227548200193182
-4.227548200177427
-4.227548200169549
-4.227548200165609
-4.227548200163641
-4.227548200162656
-4.227548200162163
-4.227548200161917
-4.2275482001617934
-4.227548200161732
-4.227548200161702
-4.227548200161687
-4.227548200161679
-4.227548200161674
-4.227548200161673
-4.227548200161672
-4.2275482001